# 01 - Kraken Connector Exploration

**Goal**: Explore the Kraken public API to inform our production connector.

**Scope**:
- Fetch ticker/orderbook data via `python-kraken-sdk`
- Map Kraken symbols (XBT) to canonical pairs (BTC)
- Parse responses into our `TopOfBook` dataclass
- Document rate limits, error handling, edge cases

**Supported Pairs** (from PROJECT_INSTRUCTIONS.md):
- BTC/USD, BTC/USDC
- LTC/USD, LTC/USDC, LTC/BTC
- SOL/USD, SOL/USDC, SOL/BTC

## 1. Setup

In [ ]:
# Install dependencies (run once)
# !pip install python-kraken-sdk httpx

In [ ]:
import asyncio
import time
from decimal import Decimal
from pprint import pprint

# Kraken SDK - we'll explore what's available
from kraken.spot import Market

# Our existing infrastructure
import sys
sys.path.insert(0, '../src')  # Adjust path as needed

from uscryptoarb.marketdata.topofbook import TopOfBook, tob_from_raw
from uscryptoarb.misc.decimals import to_decimal
from uscryptoarb.validation.guards import is_missing, require_present, require_positive

## 2. Symbol Mapping

Kraken uses non-standard symbols:
- `XBT` instead of `BTC`
- Various suffixes and prefixes

Let's explore their asset pairs endpoint first.

In [ ]:
# Create market client (no auth needed for public endpoints)
market = Market()

In [ ]:
# Fetch all tradeable asset pairs
asset_pairs = market.get_asset_pairs()
print(f"Total pairs available: {len(asset_pairs)}")
print(f"\nSample keys: {list(asset_pairs.keys())[:10]}")

In [ ]:
# Look at BTC/USD specifically - Kraken calls it XXBTZUSD or XBTUSD
btc_usd_keys = [k for k in asset_pairs.keys() if 'XBT' in k and 'USD' in k]
print("BTC/USD related pairs:")
for k in btc_usd_keys:
    print(f"  {k}")

In [ ]:
# Examine one pair's structure
if 'XXBTZUSD' in asset_pairs:
    print("XXBTZUSD details:")
    pprint(asset_pairs['XXBTZUSD'])
elif 'XBTUSD' in asset_pairs:
    print("XBTUSD details:")
    pprint(asset_pairs['XBTUSD'])

In [ ]:
# Find our target pairs
# Canonical -> Kraken symbol mapping
TARGET_PAIRS = [
    "BTC/USD", "BTC/USDC",
    "LTC/USD", "LTC/USDC", "LTC/BTC",
    "SOL/USD", "SOL/USDC", "SOL/BTC",
]

# Search patterns - Kraken uses various formats
def find_kraken_symbol(canonical: str, pairs: dict) -> str | None:
    """Try to find Kraken's symbol for a canonical pair."""
    base, quote = canonical.split('/')
    
    # BTC -> XBT mapping
    kraken_base = 'XBT' if base == 'BTC' else base
    kraken_quote = quote
    
    # Try different formats Kraken uses
    candidates = [
        f"{kraken_base}{kraken_quote}",           # XBTUSD
        f"X{kraken_base}Z{kraken_quote}",         # XXBTZUSD
        f"{kraken_base}/{kraken_quote}",          # XBT/USD (wsname format)
    ]
    
    for c in candidates:
        if c in pairs:
            return c
    
    # Check wsname field in pair info
    for k, v in pairs.items():
        wsname = v.get('wsname', '')
        if wsname == f"{kraken_base}/{kraken_quote}":
            return k
    
    return None

# Build mapping
symbol_map = {}
for canonical in TARGET_PAIRS:
    kraken_sym = find_kraken_symbol(canonical, asset_pairs)
    symbol_map[canonical] = kraken_sym
    status = "✅" if kraken_sym else "❌"
    print(f"{status} {canonical} -> {kraken_sym}")

In [ ]:
# Create the final mapping (only supported pairs)
KRAKEN_SYMBOL_MAP = {k: v for k, v in symbol_map.items() if v is not None}
print("\nFinal symbol mapping:")
pprint(KRAKEN_SYMBOL_MAP)

## 3. Fetch Ticker Data

The ticker endpoint gives us best bid/ask prices and sizes.

In [ ]:
# Fetch ticker for BTC/USD
btc_symbol = KRAKEN_SYMBOL_MAP.get('BTC/USD')
if btc_symbol:
    ticker = market.get_ticker(pair=btc_symbol)
    print(f"Ticker for {btc_symbol}:")
    pprint(ticker)

In [ ]:
# Understand the ticker structure
# Kraken ticker fields:
#   a = ask [price, whole lot volume, lot volume]
#   b = bid [price, whole lot volume, lot volume]
#   c = last trade closed [price, lot volume]
#   v = volume [today, last 24 hours]
#   p = volume weighted average price [today, last 24 hours]
#   t = number of trades [today, last 24 hours]
#   l = low [today, last 24 hours]
#   h = high [today, last 24 hours]
#   o = today's opening price

if btc_symbol and btc_symbol in ticker:
    data = ticker[btc_symbol]
    print("\nParsed ticker fields:")
    print(f"  Best Ask: price={data['a'][0]}, volume={data['a'][2]}")
    print(f"  Best Bid: price={data['b'][0]}, volume={data['b'][2]}")
    print(f"  Last Trade: price={data['c'][0]}")

In [ ]:
# Fetch multiple tickers at once (more efficient)
all_symbols = list(KRAKEN_SYMBOL_MAP.values())
all_tickers = market.get_ticker(pair=','.join(all_symbols))
print(f"Fetched {len(all_tickers)} tickers")
for sym, data in all_tickers.items():
    print(f"  {sym}: bid={data['b'][0]}, ask={data['a'][0]}")

## 4. Parse into TopOfBook

Now let's create a parser function that converts Kraken ticker data into our `TopOfBook` dataclass.

In [ ]:
def parse_kraken_ticker(
    kraken_symbol: str,
    canonical_pair: str,
    ticker_data: dict,
    ts_local_ms: int,
) -> TopOfBook:
    """
    Parse Kraken ticker response into TopOfBook.
    
    Args:
        kraken_symbol: Kraken's symbol (e.g., 'XXBTZUSD')
        canonical_pair: Our canonical pair (e.g., 'BTC/USD')
        ticker_data: The ticker dict for this symbol
        ts_local_ms: Local timestamp when data was received
        
    Returns:
        TopOfBook instance
        
    Raises:
        ValueError: If data is missing or invalid
    """
    # Kraken ticker structure:
    # a = [ask_price, whole_lot_volume, lot_volume]
    # b = [bid_price, whole_lot_volume, lot_volume]
    
    ask_array = require_present(ticker_data.get('a'), f'{kraken_symbol}.a')
    bid_array = require_present(ticker_data.get('b'), f'{kraken_symbol}.b')
    
    # Extract price and volume (index 2 is lot volume)
    ask_px = ask_array[0]
    ask_sz = ask_array[2]  # lot volume at best ask
    bid_px = bid_array[0]
    bid_sz = bid_array[2]  # lot volume at best bid
    
    return tob_from_raw(
        venue='kraken',
        pair=canonical_pair,
        ts_local_ms=ts_local_ms,
        ts_exchange_ms=None,  # Ticker doesn't include timestamp
        bid_px=bid_px,
        bid_sz=bid_sz,
        ask_px=ask_px,
        ask_sz=ask_sz,
    )

In [ ]:
# Test the parser
ts_now = int(time.time() * 1000)

for canonical, kraken_sym in KRAKEN_SYMBOL_MAP.items():
    if kraken_sym in all_tickers:
        tob = parse_kraken_ticker(
            kraken_symbol=kraken_sym,
            canonical_pair=canonical,
            ticker_data=all_tickers[kraken_sym],
            ts_local_ms=ts_now,
        )
        print(f"{tob.pair}: bid={tob.bid_px} ({tob.bid_sz}), ask={tob.ask_px} ({tob.ask_sz})")

## 5. Orderbook Depth (Alternative)

The ticker only shows top-of-book. For deeper liquidity analysis, we need the orderbook.

In [ ]:
# Fetch orderbook (depth=1 for just top of book, faster)
if btc_symbol:
    orderbook = market.get_order_book(pair=btc_symbol, count=5)  # top 5 levels
    print(f"Orderbook for {btc_symbol}:")
    pprint(orderbook)

In [ ]:
# Orderbook structure:
# asks = [[price, volume, timestamp], ...] sorted ascending by price (best ask first)
# bids = [[price, volume, timestamp], ...] sorted descending by price (best bid first)

if btc_symbol and btc_symbol in orderbook:
    book = orderbook[btc_symbol]
    print("\nTop 3 Asks (sell orders):")
    for level in book['asks'][:3]:
        print(f"  Price: {level[0]}, Size: {level[1]}, Time: {level[2]}")
    
    print("\nTop 3 Bids (buy orders):")
    for level in book['bids'][:3]:
        print(f"  Price: {level[0]}, Size: {level[1]}, Time: {level[2]}")

In [ ]:
def parse_kraken_orderbook(
    kraken_symbol: str,
    canonical_pair: str,
    book_data: dict,
    ts_local_ms: int,
) -> TopOfBook:
    """
    Parse Kraken orderbook response into TopOfBook.
    
    Uses the orderbook instead of ticker - includes exchange timestamp.
    """
    asks = require_present(book_data.get('asks'), f'{kraken_symbol}.asks')
    bids = require_present(book_data.get('bids'), f'{kraken_symbol}.bids')
    
    if not asks or not bids:
        raise ValueError(f"Empty orderbook for {kraken_symbol}")
    
    # Best ask is first in asks array (lowest price)
    # Best bid is first in bids array (highest price)
    best_ask = asks[0]  # [price, volume, timestamp]
    best_bid = bids[0]
    
    # Kraken orderbook timestamps are Unix seconds (float)
    # Use the more recent of bid/ask timestamps
    ts_exchange_sec = max(float(best_ask[2]), float(best_bid[2]))
    ts_exchange_ms = int(ts_exchange_sec * 1000)
    
    return tob_from_raw(
        venue='kraken',
        pair=canonical_pair,
        ts_local_ms=ts_local_ms,
        ts_exchange_ms=ts_exchange_ms,
        bid_px=best_bid[0],
        bid_sz=best_bid[1],
        ask_px=best_ask[0],
        ask_sz=best_ask[1],
    )

In [ ]:
# Test orderbook parser
if btc_symbol and btc_symbol in orderbook:
    tob = parse_kraken_orderbook(
        kraken_symbol=btc_symbol,
        canonical_pair='BTC/USD',
        book_data=orderbook[btc_symbol],
        ts_local_ms=int(time.time() * 1000),
    )
    print(f"From orderbook: {tob}")
    print(f"Exchange timestamp: {tob.ts_exchange_ms}")

## 6. Rate Limits & Error Handling

Kraken has rate limits we need to respect.

In [ ]:
# Kraken rate limits (from docs):
# - Public endpoints: ~1 request/second sustained
# - Burst allowed but will hit limits quickly
# - Rate limit counter decreases by 1 every 3 seconds

# Test rapid requests to see behavior
import time

results = []
errors = []

for i in range(5):
    try:
        start = time.time()
        ticker = market.get_ticker(pair=btc_symbol)
        elapsed = time.time() - start
        results.append(elapsed)
        print(f"Request {i+1}: {elapsed:.3f}s")
    except Exception as e:
        errors.append(str(e))
        print(f"Request {i+1}: ERROR - {e}")
    time.sleep(0.5)  # Small delay between requests

if results:
    print(f"\nAvg latency: {sum(results)/len(results):.3f}s")
if errors:
    print(f"Errors: {errors}")

## 7. Async Client (Production Pattern)

For the production connector, we'll use async. Let's explore the async API.

In [ ]:
# The SDK has async support
from kraken.spot import Market as AsyncMarket

async def fetch_all_tickers_async(symbols: list[str]) -> dict:
    """Fetch tickers using async client."""
    market = AsyncMarket()
    # The SDK's get_ticker is actually synchronous
    # For true async, we'd use httpx directly
    return market.get_ticker(pair=','.join(symbols))

# Note: The python-kraken-sdk's Market class is synchronous
# For production, we have two options:
# 1. Use the SDK in a thread pool (asyncio.to_thread)
# 2. Use httpx directly for true async

print("SDK is synchronous - will need httpx wrapper for true async")

In [ ]:
# Direct httpx approach (what we'll use in production)
import httpx

KRAKEN_API_URL = "https://api.kraken.com"

async def fetch_ticker_httpx(client: httpx.AsyncClient, pair: str) -> dict:
    """Fetch ticker using httpx (true async)."""
    url = f"{KRAKEN_API_URL}/0/public/Ticker"
    response = await client.get(url, params={'pair': pair})
    response.raise_for_status()
    data = response.json()
    
    # Kraken wraps responses in {error: [], result: {...}}
    if data.get('error'):
        raise ValueError(f"Kraken API error: {data['error']}")
    
    return data['result']

async def demo_httpx():
    async with httpx.AsyncClient(timeout=10.0) as client:
        symbols = ','.join(KRAKEN_SYMBOL_MAP.values())
        result = await fetch_ticker_httpx(client, symbols)
        return result

# Run async demo
httpx_result = await demo_httpx()
print(f"Fetched {len(httpx_result)} tickers via httpx")
for sym, data in list(httpx_result.items())[:3]:
    print(f"  {sym}: bid={data['b'][0]}")

## 8. Summary & Findings

### Symbol Mapping
- Kraken uses `XBT` for Bitcoin, not `BTC`
- Pairs can have prefixes: `XXBTZUSD` vs `XBTUSD`
- Use `wsname` field from asset_pairs for reliable mapping

### Data Sources
| Endpoint | Use Case | Latency | Includes Timestamp |
|----------|----------|---------|-------------------|
| `/Ticker` | Quick BBO snapshot | ~100-200ms | No |
| `/Depth` | Orderbook with depth | ~100-200ms | Yes (per level) |

### Rate Limits
- ~1 req/sec sustained for public endpoints
- Recommend 500ms minimum between requests
- Batch multiple pairs in single request

### Production Connector Design
```python
# Recommended structure:
connectors/kraken/
    __init__.py
    client.py      # httpx-based async client
    parser.py      # parse_ticker(), parse_orderbook() -> TopOfBook
    symbols.py     # SYMBOL_MAP, canonical_to_kraken(), kraken_to_canonical()
```

In [ ]:
# Final symbol map for production
print("KRAKEN_SYMBOL_MAP = {")
for canonical, kraken in KRAKEN_SYMBOL_MAP.items():
    print(f'    "{canonical}": "{kraken}",')
print("}")

In [ ]:
# Reverse map (for parsing responses back to canonical)
KRAKEN_TO_CANONICAL = {v: k for k, v in KRAKEN_SYMBOL_MAP.items()}
print("\nKRAKEN_TO_CANONICAL = {")
for kraken, canonical in KRAKEN_TO_CANONICAL.items():
    print(f'    "{kraken}": "{canonical}",')
print("}")